### Урок 2. Keras#

### -- Автор: Шенк Евгений Станиславович

### Задание 1.

<ol>
    <li>Попробуйте обучить, нейронную сеть на Keras(рассмотренную на уроке) на датасете MNIST с другими параметрами. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?</li>
</ol>

Изначальные параметры:

Количество нейронов между внутренними слоями = 64,  
Активация первых двух слоев = 'relu',  
Активация последнего слоя = 'softmax' (подходит для данной ситуации лучше всего и ее не менял)  
Оптимизатор = 'adam',  
loss = 'categorical_crossentropy'  (не изменял)
epochs = 5.  
batch_size = 32.  

При таких данных результат 0.9655

params = [[64, 'relu',  64, 'relu', 10, 'softmax'], ['adam', 'categorical_crossentropy'], [5, 32]]

Изменения оптимизатора и метода активации первых двух слоев сильного прироста не давали, хотя некоторые виды неподходили вовсе. При изменении только 'relu' на 'elu' результат accuracy: 0.9730, а при изменениии только 'adam' на 'Adadelta' результат accuracy: 0.9717.  

Увеличение количества нейронов между внутренними слоями с 64 до 128 улучшело результат до accuracy: 0.9725
Уменьшение - ухудшало результат: 32 - accuracy: ~0.95; 16 - accuracy: ~0.93

Увеличение количества эпох улучшает результат, но увеличивает затраты времени на обучение и наоборот.

Увеличение размера батча ускоряет процесс обучения, но ухудшает результат. Попытка поставить batch_size = 1 привела к увеличению затрат времени в 50 раз и ухудшению результата accuracy: ~0.92. После нескольких экспериментов изначальное значение batch_size = 32 оказалось оптимальным.

Лучший результат при приемлемом времени получился (accuracy: 0.9951):  
Количество нейронов между внутренними слоями = 128,  
Активация первых двух слоев = 'elu',  
Активация последнего слоя = 'softmax'  
Оптимизатор = 'Adadelta',  
loss = 'categorical_crossentropy'  (не изменял)  
epochs = 15.  
batch_size = 32.  

params = [[128, 'elu',  128, 'elu', 10, 'softmax'], ['Adadelta', 'categorical_crossentropy'], [15, 32]]

изменение batch_size 32 -> 128 accuracy: 0.9951 -> 0.9923, но отработало раза в 3 быстрее

In [1]:
import numpy as np
import keras
import mnist
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [2]:
np.random.seed(2177)
tf.random.set_seed(2177)

In [3]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [4]:
params = [[128, 'elu',  128, 'elu', 10, 'softmax'], ['Adadelta', 'categorical_crossentropy'], [15, 32]] # accuracy: 0.9951

# 'relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential'
# 'SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl' 

# softplus(x) = log(exp(x) + 1) # accuracy: 0.9710
# tanh # accuracy: 0.9701
# selu # accuracy: 0.9701
# elu # accuracy: 0.9730

# Adadelta #accuracy: 0.9717

# epoch=7 # accuracy: 0.9721
# epoch=15 # accuracy: 0.9852

# params = [[64, 'relu',  64, 'relu', 10, 'softmax'], ['adam', 'categorical_crossentropy'], [5, 32]] # accuracy: 0.9655

# params = [[64, 'elu',  64, 'elu', 10, 'softmax'], [opt, 'categorical_crossentropy'], [5, 32]]

# params = [[64, 'relu', 64, 'relu', 10, 'softmax'], ['adam', 'categorical_crossentropy'], [5, 32]] # accuracy: 0.9733

In [5]:
"""
opt = tf.keras.optimizers.Ftrl(
    learning_rate=0.001,
    learning_rate_power=-0.5,
    initial_accumulator_value=0.1,
    l1_regularization_strength=0.0,
    l2_regularization_strength=0.0,
    name="Ftrl",
    l2_shrinkage_regularization_strength=0.0
)
"""

'\nopt = tf.keras.optimizers.Ftrl(\n    learning_rate=0.001,\n    learning_rate_power=-0.5,\n    initial_accumulator_value=0.1,\n    l1_regularization_strength=0.0,\n    l2_regularization_strength=0.0,\n    name="Ftrl",\n    l2_shrinkage_regularization_strength=0.0\n)\n'

In [6]:
# Build the model.
model = Sequential([
  Dense(params[0][0], activation=params[0][1], input_shape=(784,)),
  Dense(params[0][2], activation=params[0][3]),
  Dense(params[0][4], activation=params[0][5]),
])

# Compile the model.
model.compile(
  optimizer=params[1][0],
  loss=params[1][1],
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=params[2][0],
  batch_size=params[2][1]
)

# Evaluate the model.
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

# Save the model to disk.
# model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

Epoch 1/15
60000/60000 [==============================] - 2s 36us/step - loss: 0.3128 - accuracy: 0.9023
Epoch 2/15
60000/60000 [==============================] - 2s 34us/step - loss: 0.1365 - accuracy: 0.9580
Epoch 3/15
60000/60000 [==============================] - 2s 34us/step - loss: 0.0981 - accuracy: 0.9693
Epoch 4/15
60000/60000 [==============================] - 2s 35us/step - loss: 0.0771 - accuracy: 0.9764
Epoch 5/15
60000/60000 [==============================] - 2s 34us/step - loss: 0.0636 - accuracy: 0.9799
Epoch 6/15
60000/60000 [==============================] - 2s 34us/step - loss: 0.0531 - accuracy: 0.9832
Epoch 7/15
60000/60000 [==============================] - 2s 36us/step - loss: 0.0459 - accuracy: 0.9854
Epoch 8/15
60000/60000 [==============================] - 2s 35us/step - loss: 0.0400 - accuracy: 0.9870
Epoch 9/15
60000/60000 [==============================] - 2s 34us/step - loss: 0.0335 - accuracy: 0.9889
Epoch 10/15
60000/60000 [==============================

In [7]:
# Predict on the first 5 test images.
predictions = model.predict(test_images[:50])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:50]) # [7, 2, 1, 0, 4]

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4]
